In [13]:
import os
import json
import pandas as pd
import nltk
import openai
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from dotenv import load_dotenv
from typing import List, Dict, Tuple
from collections import Counter

# 📦 Setup
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')
nltk.download('punkt')
nltk.download('stopwords')

# 🎭 Allowed Emotion Categories
EMOTION_CATEGORIES = ['surprised', 'happy', 'angry', 'fearful', 'disgust', 'sad', 'neutral']
SENTIMENT_CATEGORIES = ['positive', 'negative', 'neutral']

# 😃 Emoji emotion/sentiment map (for future expansion or fallback use)
EMOJI_SENTIMENTS = {
    '😊': ('positive', 'happy'), '😁': ('positive', 'happy'), '😍': ('positive', 'happy'),
    '👍': ('positive', 'happy'), '😢': ('negative', 'sad'), '😡': ('negative', 'angry'),
    '👎': ('negative', 'disgust'), '😭': ('negative', 'sad'),
    '😱': ('neutral', 'surprised'), '😨': ('negative', 'fearful'), '😐': ('neutral', 'neutral'),
    '🚚': ('neutral', 'neutral'), '🔋': ('neutral', 'neutral'), '📞': ('neutral', 'neutral')
}

# 🔍 Extract keywords and emojis
def extract_keywords_emojis(text: str) -> Tuple[List[str], Dict[str, Tuple[str, str]]]:
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    keywords = [word for word in words if word.isalnum() and word not in stop_words]
    emojis = {char: EMOJI_SENTIMENTS[char] for char in text if char in EMOJI_SENTIMENTS}
    return keywords, emojis

# 🤖 GPT-based analysis
def analyze_feedback(text: str) -> Dict:
    keywords, emojis = extract_keywords_emojis(text)
    try:
        prompt = f"""
        You are an expert in customer review analysis. Analyze the following customer feedback:

        "{text}"

        Return a JSON object with EXACTLY the following keys:
        - "sentiment": one of ["positive", "negative", "neutral"]
        - "sentiment_confidence": a number from 0 to 100 representing the confidence percentage
        - "emotion": one of ["surprised", "happy", "angry", "fearful", "disgust", "sad", "neutral"]
        - "issues": a list of main issues mentioned
        - "suggested_solutions": a list of suggested actions or responses
        - "category": a high-level tag like "delivery", "support", "quality", etc.

        Ensure all fields are included even if some values are empty.
        """

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that provides structured analysis of customer reviews."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=300
        )

        result = json.loads(response.choices[0].message.content)

    except Exception:
        # Fallback values
        result = {
            "sentiment": "neutral",
            "sentiment_confidence": 50,
            "emotion": "neutral",
            "issues": ["unspecified"],
            "suggested_solutions": ["review needed"],
            "category": "general"
        }

    return {
        "text": text,
        "sentiment": result.get("sentiment", "neutral"),
        "sentiment_confidence": result.get("sentiment_confidence", 50),
        "emotion": result.get("emotion", "neutral"),
        "issues": result.get("issues", []) or [],
        "suggested_solutions": result.get("suggested_solutions", []) or [],
        "category": result.get("category", "general"),
        "keywords": keywords,
        "emojis": list(emojis.keys())
    }

# 📁 CSV feedback analysis
def process_feedback_csv(csv_path: str, feedback_col: str = "Review") -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    if feedback_col not in df.columns:
        raise ValueError(f"Column '{feedback_col}' not found in CSV")

    results = []
    sentiment_counter = Counter()
    emotion_counter = Counter()

    for feedback in df[feedback_col].dropna():
        result = analyze_feedback(str(feedback))
        sentiment_counter[result["sentiment"]] += 1
        emotion_counter[result["emotion"]] += 1

        results.append({
            "Feedback": result["text"],
            "Sentiment": result["sentiment"],
            "Sentiment Confidence (%)": result["sentiment_confidence"],
            "Emotion": result["emotion"],
            "Category": result["category"],
            "Issues": ", ".join(result["issues"]),
            "Solutions": ", ".join(result["suggested_solutions"]),
            "Emojis": ", ".join(result["emojis"]),
            "Keywords": ", ".join(result["keywords"])
        })

    processed_df = pd.DataFrame(results)

    # 📊 Sentiment Percentage Summary
    total = sum(sentiment_counter.values())
    print("\n📊 Sentiment Breakdown:")
    for sentiment in SENTIMENT_CATEGORIES:
        percent = (sentiment_counter[sentiment] / total * 100) if total else 0
        print(f"{sentiment.capitalize()}: {percent:.2f}%")

    # 🎭 Emotion Category Summary
    print("\n🎭 Emotion Breakdown:")
    for emotion in EMOTION_CATEGORIES:
        print(f"{emotion.capitalize()}: {emotion_counter[emotion]}")

    return processed_df


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sivaj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sivaj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
df = process_feedback_csv("D:/faite/sample_feedbacks.csv", feedback_col="feedback")




📊 Sentiment Breakdown:
Positive: 50.00%
Negative: 50.00%
Neutral: 0.00%

🎭 Emotion Breakdown:
Surprised: 0
Happy: 5
Angry: 4
Fearful: 0
Disgust: 1
Sad: 0
Neutral: 0


In [17]:
df

,Feedback,Sentiment,Sentiment Confidence (%),Emotion,Category,Issues,Solutions,Emojis,Keywords
0,The delivery was late and the packaging was da...,negative,100,angry,delivery,"late delivery, damaged packaging","Improve delivery timelines, Enhance packaging ...",,"delivery, late, packaging, damaged"
1,Excellent customer service! I'm very satisfied.,positive,95,happy,customer service,,,,"excellent, customer, service, satisfied"
2,Battery life is poor and needs improvement.,negative,80,disgust,product quality,poor battery life,improve battery life,,"battery, life, poor, needs, improvement"
3,Shipping was fast and the product arrived safely.,positive,100,happy,delivery,,,,"shipping, fast, product, arrived, safely"
4,"Product quality is terrible, I want a refund.",negative,90,angry,quality,product quality,refund,,"product, quality, terrible, want, refund"
5,Very happy with the quick response from support.,positive,90,happy,support,,,,"happy, quick, response, support"
6,Item was broken on arrival and support was unh...,negative,100,angry,quality,"broken item on arrival, unhelpful support","improve packaging for better protection, enhan...",,"item, broken, arrival, support, unhelpful"
7,Love the performance of this device. Great job!,positive,95,happy,product performance,,,,"love, performance, device, great, job"
8,Customer service is slow and unresponsive.,negative,100,angry,customer service,"slow customer service, unresponsive customer s...","Improve response times, Enhance customer servi...",,"customer, service, slow, unresponsive"
9,Really appreciate the fast delivery and qualit...,positive,95,happy,delivery,,,,"really, appreciate, fast, delivery, quality, p..."


In [19]:
df = process_feedback_csv( "D:/faite/reviews_100_with_date.csv", feedback_col="Review")



📊 Sentiment Breakdown:
Positive: 22.00%
Negative: 59.00%
Neutral: 19.00%

🎭 Emotion Breakdown:
Surprised: 0
Happy: 22
Angry: 32
Fearful: 0
Disgust: 2
Sad: 22
Neutral: 20


In [21]:
df

,Feedback,Sentiment,Sentiment Confidence (%),Emotion,Category,Issues,Solutions,Emojis,Keywords
0,Absolutely love it! 😊💯,positive,100,happy,general,,,😊,"absolutely, love"
1,Delivery took too long 🚚👎,negative,80,angry,delivery,Delivery took too long,Improve delivery speed,"🚚, 👎","delivery, took, long"
2,Late delivery and no tracking ⏰,negative,80,angry,delivery,"Late delivery, No tracking","Improve delivery time estimation, Provide trac...",,"late, delivery, tracking"
3,Too slow and bad experience 😤,negative,90,angry,service,slow service,improve speed of service,,"slow, bad, experience"
4,Good quality and fast shipping 🚚,positive,95,happy,delivery,,,🚚,"good, quality, fast, shipping"
...,...,...,...,...,...,...,...,...,...
95,Too slow and bad experience 😤,negative,90,angry,customer service,slow service,Improving speed of service,,"slow, bad, experience"
96,Battery life is too short 😢,negative,80,sad,product quality,Battery life too short,"Provide better battery life, Offer battery rep...",😢,"battery, life, short"
97,Late delivery and no tracking ⏰,negative,80,angry,delivery,"Late delivery, No tracking","Improve delivery time estimation, Provide trac...",,"late, delivery, tracking"
98,Absolutely love it! 😊💯,positive,100,happy,general,,,😊,"absolutely, love"


In [23]:
df1 = process_feedback_csv( "D:/faite/sample_reviews (1).csv", feedback_col="Review")



📊 Sentiment Breakdown:
Positive: 26.67%
Negative: 20.00%
Neutral: 53.33%

🎭 Emotion Breakdown:
Surprised: 1
Happy: 3
Angry: 1
Fearful: 0
Disgust: 1
Sad: 1
Neutral: 7


In [25]:
df1

,Feedback,Sentiment,Sentiment Confidence (%),Emotion,Category,Issues,Solutions,Emojis,Keywords
0,I love this product! 😊 It's amazing and works ...,neutral,50,neutral,general,unspecified,review needed,😊,"love, product, amazing, works, perfectly"
1,Terrible service. I'm very disappointed. 😡,neutral,50,neutral,general,unspecified,review needed,😡,"terrible, service, disappointed"
2,The delivery was late but the item was fine.,neutral,50,,delivery,late delivery,improved delivery time estimation,,"delivery, late, item, fine"
3,Fantastic support team! 😍 They helped me a lot.,positive,90,happy,support,,,😍,"fantastic, support, team, helped, lot"
4,Battery drains too fast. Not happy at all. 🔋,neutral,50,neutral,general,unspecified,review needed,🔋,"battery, drains, fast, happy"
5,Excellent packaging and fast delivery. 👍,neutral,50,neutral,general,unspecified,review needed,👍,"excellent, packaging, fast, delivery"
6,Item was broken on arrival. Very bad experienc...,negative,95,angry,quality,broken item,"Provide a replacement item immediately, Improv...",👎,"item, broken, arrival, bad, experience"
7,Neutral experience. Nothing special.,neutral,50,neutral,,,,,"neutral, experience, nothing, special"
8,I am fearful about using this again after my l...,neutral,50,neutral,general,unspecified,review needed,,"fearful, using, last, experience"
9,Product quality is bad. 😢 I want a refund.,negative,100,sad,quality,product quality,refund,😢,"product, quality, bad, want, refund"
